# parsing_tir

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm import trange, tqdm

импорт библиотек

In [2]:
reqx = requests.get('https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=2')

In [3]:
soup = bs(reqx.text, 'html.parser')

In [4]:
element = soup.findAll('div', class_='item_info')

In [5]:
element

[<div class="item_info">
 <div class="item_info--top_block">
 <div class="rating">
 <div class="blog-info__rating--top-info">
 <div class="votes_block nstar with-text">
 <div class="ratings">
 <div class="inner_rating" title="Нет оценок">
 <div class="item-rating"><i aria-hidden="true" class="svg inline svg-inline-star"><svg height="13" viewbox="0 0 15 13" width="15" xmlns="http://www.w3.org/2000/svg"><rect class="sscls-1" height="13" width="15"></rect><path class="sscls-2" d="M1333.37,457.5l-4.21,2.408,0.11,0.346,2.07,4.745h-0.72l-4.12-3-4.09,3h-0.75l2.04-4.707,0.12-.395-4.19-2.4V457h5.12l1.53-5h0.38l1.57,5h5.14v0.5Z" data-name="Shape 921 copy 15" transform="translate(-1319 -452)"></path></svg></i></div>
 <div class="item-rating"><i aria-hidden="true" class="svg inline svg-inline-star"><svg height="13" viewbox="0 0 15 13" width="15" xmlns="http://www.w3.org/2000/svg"><rect class="sscls-1" height="13" width="15"></rect><path class="sscls-2" d="M1333.37,457.5l-4.21,2.408,0.11,0.346,2.07

In [6]:
title = element[7].find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip()

создаем переменную с наименованием товара

In [7]:
urls = 'https://tiraet.com' + element[7].find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href']

In [8]:
counts = element[0].find('div', class_='item-stock js-show-stores js-show-info-block').text.strip()

In [9]:
art = int(element[0].find('div', class_='article_block').text.replace('Арт.: ',''))

In [10]:
price = float(element[7].find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ',''))

In [11]:
old_price = float(element[7].find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.')) if str(type(element[7].find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else 0

In [12]:
datе = pd.to_datetime('today').normalize()

In [13]:
data = []
for i in element:
    data.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                 'urls' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],  
                 'datе' : pd.to_datetime('today').normalize(),                
                 'counts' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                 'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')),
                 'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN                
                })

In [14]:
data = pd.DataFrame(data)
data

,title,urls,datе,counts,id,price,old_price
0,Фотоаппарат мгновенной печати INSTANT W/10SH G...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,86320,2149.0,NaN
1,Фотоаппарат мгновенной печати INSTANT W/10SH G...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,86319,2149.0,NaN
2,Объектив CANON EF 50mm f/1.8 STM (0570C005AA),https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Достаточно,42951,2211.0,NaN
3,Объектив CANON EF-S 24mm F/2.8 STM (9522B005AA),https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,50497,2310.0,NaN
4,Адаптер Canon MT ADAPTER EF-EOS R (2971C005AA...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,71429,2359.5,NaN
5,"Диктофон OLYMPUS WS-883 (цифр, 8Gb, mSD, USB, ...",https://tiraet.com/catalog/PHoto--i-video/audi...,2024-06-04,Мало,81315,2409.0,NaN
6,Фотоаппарат мгновенной печати INSTAX SQUARE SQ...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,85767,2599.1,NaN
7,Фотоаппарат мгновенной печати INSTAX SQUARE SQ...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,85769,2599.1,NaN
8,Фотоаппарат мгновенной печати INSTAX SQUARE SQ...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,84068,2599.1,NaN
9,Фотоаппарат мгновенной печати INSTAX SQUARE SQ...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,86321,3229.1,NaN


In [15]:
data.columns

Index(['title', 'urls', 'datе', 'counts', 'id', 'price', 'old_price'], dtype='object')

In [16]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [26]:
url = ['https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=']

In [34]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'urls', 'datе', 'counts', 'id', 'price', 'old_price'])
    for u in tqdm(url):        
        for j in range(1,3):        
            reqx = requests.get(u + str(j))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'urls' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'datе' : pd.to_datetime('today').normalize(),
                             'counts' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')),
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

In [35]:
df = parsing_2(url)
df

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.73s/it]


,title,urls,datе,counts,id,price,old_price
0,Чехол наручный MP3A-BAG1 для MP3-плееров,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Достаточно,11287,16.3,NaN
1,Сумка-футляр для ф/видео Sumdex POC-471OG оран...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Много,33161,16.3,NaN
2,Сумка-футляр для MP3-Player BAG-ZG-EMP-31 ZIGN...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,30729,16.3,NaN
3,Сумка-футляр для ф/видео Sumdex POC-476BK черн...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,33167,99.0,NaN
4,Сумка-футляр для ф/видео Sumdex POC-476SA сини...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,33169,99.0,NaN
5,Фотоштатив Maclean Tripod (MC-164) для смартфо...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,84295,159.1,NaN
6,Сумка-футляр для ф/видео ACROPOLIS CB-19 черная,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,37890,164.0,NaN
7,Светильник светодиодный для видео и фото Natec...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Много,74413,299.0,NaN
8,"MP3-плеер 8Mb Sencor (SFP 4408BK) 1.1' ЖК, FM-...",https://tiraet.com/catalog/PHoto--i-video/audi...,2024-06-04,Мало,76357,499.0,NaN
9,Фотоштатив Hama Tripod Star 75 (арт.4175) высо...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,76348,529.0,NaN


In [ ]:
flex-direction-nav

In [46]:
max_ = soup.findAll('div', class_='nums')

In [41]:
max_ = element.find('ui', class_='flex-direction-nav')

In [67]:
max_[4].find('a', class_='dark_link')

IndexError: list index out of range

In [56]:
max_

[<div class="nums">
 <ul class="flex-direction-nav">
 <li class="flex-nav-prev colored_theme_hover_text">
 <a class="flex-prev" href="/catalog/PHoto--i-video/">
 <i aria-hidden="true" class="svg svg-inline-down"><svg height="10" viewbox="0 0 5.97 10" width="5.97" xmlns="http://www.w3.org/2000/svg"><path class="cls-1" d="M1302.98,2891a0.972,0.972,0,0,1-.23.62c-0.02.03-.03,0.06-0.05,0.09h-0.01l-4.03,3.99a0.955,0.955,0,0,1-1.39,0,1.014,1.014,0,0,1,0-1.41l3.32-3.29-3.3-3.32a0.99,0.99,0,0,1,1.4-1.4l3.98,3.99a0.077,0.077,0,0,0,.03.02A1.031,1.031,0,0,1,1302.98,2891Z" data-name="Rounded Rectangle 996" transform="translate(-1297 -2886)"></path></svg></i> </a>
 </li>
 <li class="flex-nav-next colored_theme_hover_text">
 <a class="flex-next" href="/catalog/PHoto--i-video/?PAGEN_1=3">
 <i aria-hidden="true" class="svg svg-inline-down"><svg height="10" viewbox="0 0 5.97 10" width="5.97" xmlns="http://www.w3.org/2000/svg"><path class="cls-1" d="M1302.98,2891a0.972,0.972,0,0,1-.23.62c-0.02.03-.03,0.0

In [36]:
preprocessing(df)

таблица имеет следующий вид:


,title,urls,datе,counts,id,price,old_price
0,Чехол наручный MP3A-BAG1 для MP3-плееров,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Достаточно,11287,16.3,NaN
1,Сумка-футляр для ф/видео Sumdex POC-471OG оран...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Много,33161,16.3,NaN
2,Сумка-футляр для MP3-Player BAG-ZG-EMP-31 ZIGN...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,30729,16.3,NaN
3,Сумка-футляр для ф/видео Sumdex POC-476BK черн...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,33167,99.0,NaN
4,Сумка-футляр для ф/видео Sumdex POC-476SA сини...,https://tiraet.com/catalog/PHoto--i-video/phot...,2024-06-04,Мало,33169,99.0,NaN


****************************************************************************************************
таблица имеет 40 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


title         0
urls          0
datе          0
counts        0
id            0
price         0
old_price    39
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


title         0.0
urls          0.0
datе          0.0
counts        0.0
id            0.0
price         0.0
old_price    97.5
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


title                object
urls                 object
datе         datetime64[ns]
counts               object
id                   object
price               float64
old_price           float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   title      40 non-null     object        
 1   urls       40 non-null     object        
 2   datе       40 non-null     datetime64[ns]
 3   counts     40 non-null     object        
 4   id         40 non-null     object        
 5   price      40 non-null     float64       
 6   old_price  1 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 2.3+ KB


None